In [2]:
# As usual, a bit of setup
import time
from time import process_time
import numpy as np
import matplotlib.pyplot as plt
import LOUPE.WILLOW.loupe as lp
import tensorflow as tf
import h5py
import pandas as pd
import csv
import copy
import math
from utils.data_utils import *
import sys
import re
from utils.spj import Config
from utils.spj import SPJ
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

home_dir = "/home/martnzjulio_a/songze"
#home_dir = "/home/songzeli"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def model(all_train, all_val, learning_rate, num_epochs, minibatch_size, num_steps, print_cost = True):
    """
    Implements a tensorflow neural network: C3D->ATTENTION->CAPTIONING
    
    Arguments:
    H_train -- training set, of shape = [n_train,num_c3d_features,num_proposals]
    Y_train -- caption labels, of shape = [n_train,num_proposals,num_steps+1]
    H_test -- training set, of shape = [n_test,num_c3d_features,num_proposals]
    Y_test -- caption labels, of shape = [n_test,num_proposals,num_steps+1]
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    (H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train) = all_train
    (H_val,   Ipast_val,   Ifuture_val,   Ycaptions_val,   Xcaptions_val)   = all_val
    
    
    # Directory to Save Checkpoint
    checkpoint_dir = home_dir + "/checkpoints/"
    tensorboard_dir =  home_dir + "/tensorboard/"
    
    # Reset Graph
    tf.reset_default_graph()    
    
    # For Consistency
    tf.set_random_seed(1)                             
    seed = 3                                         
    
    # Number of Training Examples
    num_train = H_train.shape[0] 
    num_val = H_val.shape[0] 
    
    # to keep track of costs
    costs = []
    
    # Global Epoch Number
    global_step = tf.Variable(0, name='global_step', trainable=False)
    
    # Model
    config = Config()
    spj = SPJ(config)
        
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(spj.loss, global_step=global_step)
    
    # Initialize all the variables
    init = tf.global_variables_initializer()
    
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()
    
    # Tensorboard Loss
    training_summary = tf.summary.scalar("training_loss", spj.loss)
    validation_summary = tf.summary.scalar("validation_loss", spj.loss)
    

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
    
        # check for latest checkpoint
        latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
        if latest_checkpoint == None:
            # If no check point run the initialization
            print()
            print("No checkpoint exists, initializing parameters...")
            sess.run(init)
        else:
            print()
            print("Restoring from latest checkpoint...")
            saver.restore(sess, latest_checkpoint)
        
        #Tensorboard
        summary_writer = tf.summary.FileWriter(tensorboard_dir,sess.graph)
        
        # Training Loop
        print()
        start = process_time() # Record start time
        for epoch in range(num_epochs):
            #print(epoch)
            
            # Variable to store cost
            epoch_train_loss = 0.0
            epoch_val_loss = 0.0
            
            # Get minibatches
            num_train_minibatches = num_train // minibatch_size 
            num_val_minibatches = num_val // minibatch_size 
            seed = seed + 1
            train_minibatches = random_mini_batches(H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train, minibatch_size, seed)
            val_minibatches = random_mini_batches(H_val,   Ipast_val,   Ifuture_val,   Ycaptions_val,   Xcaptions_val,   minibatch_size, seed)

            for counter, train_minibatch in enumerate(train_minibatches):
                #print(counter)
                
                # Select minibatch
                (minibatch_H_train, minibatch_Ipast_train, minibatch_Ifuture_train, minibatch_Ycaptions_train, minibatch_Xcaptions_train) = train_minibatch
                minibatch_Ycaptions_train = id_2_one_hot_void_padding(minibatch_Ycaptions_train, spj.config.num_classes, void_dim=0)
                
                # Run Train Session
                _ , minibatch_train_loss, train_summ = sess.run([optimizer, spj.loss, training_summary], feed_dict={spj._H: minibatch_H_train, spj._Ipast: minibatch_Ipast_train, spj._Ifuture: minibatch_Ifuture_train, spj._x: minibatch_Xcaptions_train, spj._y: minibatch_Ycaptions_train, spj._batchsize: minibatch_H_train.shape[0]}) #
                epoch_train_loss += minibatch_train_loss / num_train_minibatches
                
            for counter, val_minibatch in enumerate(val_minibatches):
                
                # Select minibatch
                (minibatch_H_val, minibatch_Ipast_val, minibatch_Ifuture_val, minibatch_Ycaptions_val, minibatch_Xcaptions_val) = val_minibatch
                minibatch_Ycaptions_val = id_2_one_hot_void_padding(minibatch_Ycaptions_val, spj.config.num_classes, void_dim=0)
                
                # Run Validation Session
                minibatch_val_loss, val_summ = sess.run([spj.loss, validation_summary], feed_dict={spj._H: minibatch_H_val, spj._Ipast: minibatch_Ipast_val, spj._Ifuture: minibatch_Ifuture_val, spj._x: minibatch_Xcaptions_val, spj._y: minibatch_Ycaptions_val, spj._batchsize: minibatch_H_val.shape[0]}) #
                epoch_val_loss += minibatch_val_loss / num_val_minibatches
             
            # Epoch Training Loss
            #loss_train, train_summ = sess.run([spj.loss, training_summary], feed_dict={spj._H: H_train, spj._Ipast: Ipast_train, spj._Ifuture: Ifuture_train, spj._x: Xcaptions_train, spj._y: Ycaptions_train, spj._batchsize: num_train }) 
            # Epoch Validation Session
            #loss_val, val_summ = sess.run([spj.loss, validation_summary], feed_dict={spj._H: H_val, spj._Ipast: Ipast_val, spj._Ifuture: Ifuture_val, spj._x: Xcaptions_val, spj._y: Ycaptions_val, spj._batchsize: num_val }) #    
            
            # Print cost
            if print_cost == True:
                global_epoch = tf.train.global_step(sess, global_step)//num_train_minibatches
                print("Epoch: ", global_epoch)
                print ("Training Loss: ", epoch_train_loss)
                print ("Validation Loss: ", epoch_val_loss)
                summary_writer.add_summary(train_summ, global_step=global_epoch) 
                summary_writer.add_summary(val_summ, global_step=global_epoch) 
            
            # Save Model (every 20 epochs)
            if epoch % 20 == 0:
                print("Saving Checkpoint for global_step " + str(global_epoch))
                saver.save(sess, checkpoint_dir + 'model', global_step = global_epoch)
        
        # Save and Print Processed Time
        end = process_time() 
        print()
        print("Time Elapased: ", end - start)
        
        return 0

# Load Vocab

In [4]:
# Load Vocabulary
embedding_size =512
pad_len, num_steps = 50, 50
vocabulary,vocab_size = caption_preprocess(home_dir)
emb_matrix,word2id,id2word = get_wordvector(embedding_size,vocab_size,vocabulary)
num_classes = len(word2id)

# Word Embedding Matrix
emb_matrix, word2id, id2word = get_wordvector(embedding_size,vocab_size,vocabulary) #changed by Songze

Total number of words in all captions:  504895
Vocabulary Size (Unique):  13001


# Load All Data

In [5]:
# Load All Data
all_file = home_dir + "/SPJ/train_5000.csv"
all_ids,all_data,all_padded_proposals,all_padded_framestamps = video_preprocess(home_dir, all_file)

# All Captions
all_padded_sentences,all_padded_sentences_2,all_padded_sentences_id = get_padded_sentences_id(pad_len, all_ids, all_data, word2id) 
Ycaptions_all = np.transpose(copy.deepcopy(all_padded_sentences_2),axes=(0,2,1)).astype(np.int32)
Xcaptions_all = np.transpose(copy.deepcopy(all_padded_sentences),axes=(0,2,1)).astype(np.int32)

# All Features 
H_all = all_padded_proposals.astype(np.float32)
Ipast_all = temporal_indicator(all_padded_framestamps, mode="past").astype(np.float32)
Ifuture_all = temporal_indicator(all_padded_framestamps, mode="future").astype(np.float32)

data_size = len(all_ids)
print("Number of Examples in Data Set:", data_size)

Number of Examples in Data Set: 4990


# Data Split

In [6]:
# Train, Val, Test Split
num_train = 100
num_val = 10
num_test = 10

# Training Split
Xcaptions_train = Xcaptions_all[:num_train]
Ycaptions_train = Ycaptions_all[:num_train]
H_train = H_all[:num_train]
Ipast_train = Ipast_all[:num_train]
Ifuture_train = Ifuture_all[:num_train]
print("Xcaptions_train.shape: ", Xcaptions_train.shape)
print("Ycaptions_train.shape: ", Ycaptions_train.shape)
print("H_train.shape: ", H_train.shape)
print("Ipast_train.shape: ", Ipast_train.shape)
print("Ifuture_train.shape: ", Ifuture_train.shape)
print()

# Validation Split
Xcaptions_val = Xcaptions_all[num_train:num_train+num_val]
Ycaptions_val = Ycaptions_all[num_train:num_train+num_val]
H_val = H_all[num_train:num_train+num_val]
Ipast_val = Ipast_all[num_train:num_train+num_val]
Ifuture_val = Ifuture_all[num_train:num_train+num_val]
print("Xcaptions_val.shape: ", Xcaptions_val.shape)
print("Ycaptions_val.shape: ", Ycaptions_val.shape)
print("H_val.shape: ", H_val.shape)
print("Ipast_val.shape: ", Ipast_val.shape)
print("Ifuture_val.shape: ", Ifuture_val.shape)
print()

# Testing Split
Xcaptions_test = Xcaptions_all[num_train+num_val:num_train+num_val+num_test]
Ycaptions_test = Ycaptions_all[num_train+num_val:num_train+num_val+num_test]
H_test = H_all[num_train+num_val:num_train+num_val+num_test]
Ipast_test = Ipast_all[num_train+num_val:num_train+num_val+num_test]
Ifuture_test = Ifuture_all[num_train+num_val:num_train+num_val+num_test]
print("Xcaptions_test.shape: ", Xcaptions_test.shape)
print("Ycaptions_test.shape: ", Ycaptions_test.shape)
print("H_test.shape: ", H_test.shape)
print("Ipast_test.shape: ", Ipast_test.shape)
print("Ifuture_test.shape: ", Ifuture_test.shape)

Xcaptions_train.shape:  (100, 30, 50)
Ycaptions_train.shape:  (100, 30, 51)
H_train.shape:  (100, 500, 30)
Ipast_train.shape:  (100, 30, 30)
Ifuture_train.shape:  (100, 30, 30)

Xcaptions_val.shape:  (10, 30, 50)
Ycaptions_val.shape:  (10, 30, 51)
H_val.shape:  (10, 500, 30)
Ipast_val.shape:  (10, 30, 30)
Ifuture_val.shape:  (10, 30, 30)

Xcaptions_test.shape:  (10, 30, 50)
Ycaptions_test.shape:  (10, 30, 51)
H_test.shape:  (10, 500, 30)
Ipast_test.shape:  (10, 30, 30)
Ifuture_test.shape:  (10, 30, 30)


# Execute Training

In [7]:
# Train Model
learning_rate = 0.01
num_epochs = 1000
minibatch_size = 9
all_train = (H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train)
all_val =   (H_val,   Ipast_val,   Ifuture_val,   Ycaptions_val,   Xcaptions_val)
model(all_train, all_val, learning_rate, num_epochs, minibatch_size, num_steps)


No checkpoint exists, initializing parameters...

Epoch:  1
Training Loss:  5923.818572998047
Validation Loss:  5284.800079345703
Saving Checkpoint for global_step 1
Epoch:  2
Training Loss:  4620.534828879617
Validation Loss:  5151.2564697265625
Epoch:  3
Training Loss:  4433.027418656782
Validation Loss:  5128.261535644531
Epoch:  4
Training Loss:  4278.060569069601
Validation Loss:  5089.909454345703


KeyboardInterrupt: 

# Count Number of Uknowns

In [10]:
count = 0
num_proposals = Ycaptions_train.shape[1]
for i in range(Ycaptions_train.shape[0]):
    for k in range(num_proposals):
        for word_id in Ycaptions_train[i][k]:
            if word_id == 1:
                #print(id2word[word_id])
                count = count + 1
print("number of uknowns:", count)

number of uknowns: 122


# Generate Captions

In [ ]:
learning_rate = 1e-6
minibatch_size = 9
tf.set_random_seed(1)                             
seed = 3            
from utils.data_utils import *

tf.reset_default_graph()
config = Config()
spj = SPJ(config)
global_step = tf.Variable(0, name='global_step', trainable=False)
checkpoint_dir = "/home/martnzjulio_a/songze/checkpoints/"
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
saver = tf.train.Saver()
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(spj.loss, global_step=global_step)
# Initialize all the variables
init = tf.global_variables_initializer()


train_minibatches = random_mini_batches(H_train, Ipast_train, Ifuture_train, Ycaptions_train, Xcaptions_train, minibatch_size, seed)
(minibatch_H_train, minibatch_Ipast_train, minibatch_Ifuture_train, minibatch_Ycaptions_train, minibatch_Xcaptions_train) = train_minibatches[0]   
with tf.Session() as sess:   
    sess.run(init)
    word_id = spj.caption_generation(sess,minibatch_H_train, minibatch_Ipast_train, minibatch_Ifuture_train, minibatch_Xcaptions_train, minibatch_Ycaptions_train)
print (word_id)